# AMBER2LAMMPS Examples

This notebook mirrors the README tutorial examples for the `amber_to_lammps.py` CLI and the Python API.

Assumptions:
- You are running this notebook from the repo root.
- Your Jupyter kernel has `parmed` and `numpy` installed.
- If you want to run AmberTools/PACKMOL/LAMMPS steps, those executables must be on your `PATH`.


In [ ]:
from pathlib import Path
import shlex
import shutil
import subprocess
import sys

REPO = Path.cwd()
print('Repo:', REPO)
print('Python:', sys.executable)

import numpy as np
import parmed as pmd

print('numpy:', np.__version__)
print('parmed:', pmd.__version__)

for exe in ['antechamber', 'tleap', 'packmol', 'lmp']:
    print(f"{exe}: {shutil.which(exe)}")


def run_cmd(cmd, cwd=None):
    cmd = [str(c) for c in cmd]
    print('+', ' '.join(shlex.quote(c) for c in cmd))
    subprocess.run(cmd, check=True, cwd=cwd)


## Output Directory

All example outputs are written under `notebook_outputs/` so you can rerun cells without overwriting files in the repo root.


In [ ]:
out_root = REPO / 'notebook_outputs'
out_root.mkdir(exist_ok=True)
out_root


## Example 1 (CLI): Single Topology, Single Copy

This example converts the included `epon` system.


In [ ]:
epon_dir = out_root / 'epon_cli'
epon_dir.mkdir(exist_ok=True)

cmd = [
    sys.executable, REPO / 'amber_to_lammps.py',
    'data.lammps', 'parm.lammps', REPO / 'epon.pdb',
    '-t', REPO / 'epon.prmtop',
    '-c', '1',
    '--charges', '0',
    '--verbose',
    '-b', '4.5',
    '--keep-temp',
]
run_cmd(cmd, cwd=epon_dir)

sorted(p.name for p in epon_dir.iterdir())


In [ ]:
pairs_path = epon_dir / 'pairs.txt'
print('pairs.txt exists:', pairs_path.exists())
print('
'.join(pairs_path.read_text().splitlines()[:20]))


## Example 2 (CLI): Single Topology, Multiple Copies

This example converts 10 ethanol molecules using a combined PDB (`multi_ethanol.pdb`) and a single topology (`ethanol.prmtop`).


In [ ]:
ethanol_multi_dir = out_root / 'ethanol_multi_cli'
ethanol_multi_dir.mkdir(exist_ok=True)

cmd = [
    sys.executable, REPO / 'amber_to_lammps.py',
    'data.lammps', 'parm.lammps', REPO / 'multi_ethanol.pdb',
    '-t', REPO / 'ethanol.prmtop',
    '-c', '10',
    '--charges', '0',
    '--verbose',
    '--keep-temp',
]
run_cmd(cmd, cwd=ethanol_multi_dir)

sorted(p.name for p in ethanol_multi_dir.iterdir() if p.suffix in {'.lammps', '.txt'})


## Example 3 (CLI): Mixed System (Multiple Topologies)

Important: for mixed systems, the combined PDB must contain molecules in the same order and counts as `-t` / `-c`.

This repo includes a small mixed example: `mixed_system.pdb` with `aspirin`, `benzene`, `ethanol`.


In [ ]:
mixed_dir = out_root / 'mixed_cli'
mixed_dir.mkdir(exist_ok=True)

cmd = [
    sys.executable, REPO / 'amber_to_lammps.py',
    'data.lammps', 'parm.lammps', REPO / 'mixed_system.pdb',
    '-t', REPO / 'aspirin.prmtop', REPO / 'benzene.prmtop', REPO / 'ethanol.prmtop',
    '-c', '1', '2', '5',
    '--charges', '0', '0', '0',
    '--verbose',
    '--keep-temp',
]
run_cmd(cmd, cwd=mixed_dir)

print('pairs.txt sections preview:')
print('
'.join((mixed_dir / 'pairs.txt').read_text().splitlines()[:40]))


## Example 4 (Python API): Same Conversion Engine

The Python API uses the same conversion function as the CLI. The core call is:

`amber2lammps(data_file, param_file, topologies, molecule_counts, pdb_file, charges_target, ...)`


In [ ]:
from amber_to_lammps import amber2lammps, validate_files

api_dir = out_root / 'api'
api_dir.mkdir(exist_ok=True)

topologies = [str(REPO / 'epon.prmtop')]
counts = [1]
pdb_file = str(REPO / 'epon.pdb')
charges_target = [0]

validate_files(topologies, counts, pdb_file)

amber2lammps(
    data_file=str(api_dir / 'epon_data.lammps'),
    param_file=str(api_dir / 'epon_parm.lammps'),
    topologies=topologies,
    molecule_counts=counts,
    pdb_file=pdb_file,
    charges_target=charges_target,
    buffer=4.5,
    verbose=True,
    keep_temp=False,
)

sorted(p.name for p in api_dir.iterdir())
